![figure](lab5/politecnico_h-01.png)

# **Programmable Electronics**
#### Bachelor's Degree in Electrical and Computer Engineering 

## Lab Work 5 - Finite State Machines with DataPath 

-----------------------


## Introduction ##
In this work we will implement a set of state machines with datapath using a behavioral description in VHDL. This document does not provide usage details for Vivado. Whenever you have doubts at this level, consult the Lab1 statement.


### Objectives ###

In this work we propose the implementation of three circuits that implement state machines with datapath:
* Divider circuit.
* Binary/BCD Converter.
* Digital frequency meter for low frequencies.


In the instructions below **{sources}** refers to `C:\Xilinx\EEC_EP\sources` and **{labs}** refers to `(C:\Xilinx\EEC_EP\labs)`


_______________________

## Step 1 - Divider Circuit  ##

### Step 1.1 ### 

Due to its complexity, the divider circuit cannot be synthesized automatically. In this section we will implement an 8-bit integer division algorithm. The figure below shows a 4-bit example. It can be summarized in 4 steps:

1. Double the width of the dividend with leading zeros added and place the divisor on the left side of the increased dividend.

2. Consider only the 4 leftmost digits of the dividend. If these bits correspond to a number greater than or equal to the divisor number, subtract the divisor from the dividend and place a '1' in the quotient. Otherwise, keep the dividend bits and place a '0' in the quotient.

3. Add one bit of the dividend to the previous result.

4. Repeat steps 2 and 3 until all dividend bits have been used
 

<div>
<img src="lab5/fig1.png" width=900/>
</div>
                                   
                                   
The figure also shows an outline of the required action modules. Initially the divisor is stored in the **d** register and the expanded dividend is in the **rh** and **rl** registers . With each iteration, these records are shifted left by one position. The 4 leftmost bits (**rh**) are then compared and eventually subtracted from the divisor. As the dividend register is shifted to the left, it frees up memory space that can be used to accommodate the quotient bits. In the end, the **rl** register holds all the bits of the quotient, while the **rh** register stores the remainder of the division.

The ASMD diagram of this circuit consists of four states: **idle**, **op**, **last** e **done**. AMost operations (comparison, subtraction and shift) are performed in the **op** state . Since the rest of the division should not be shifted, an additional state (last) was created that differs from the previous state (op) only by the lack of shifting.



* Start by downloading the provided source files (Github or Moodle) and saving them in `{sources}/lab5`.


* Open Vivado from the icon on your desktop or: `Start > All Programs > Xilinx Design Tools > Vivado 2022.1`. 


* Select **Create Project** to start the wizard. In the Create A New Vivado Project dialog box , select **Next**.


* Click the Browse button in the project location field of the new project form, navigate to `c:/Xilinx/EEC_EP/labs` and click **Select**. Type *lab5* in the project name field. Make sure the *Create Project Subdirectory* box is checked. Click **Next**.


* Select the **RTL Project** option in the *Project type* form and click **Next**. Make sure the *Do not specify fonts at this time* box is checked.


* In the *Default Part* form, select the Boards option and choose the **Basys3** board. Click Next and then Finish to create the Vivado project.


* In the Sources pane , select **Add Sources** and follow the necessary steps (*Add or create design sources*) to add the **div.vhd** file, which contains the VHDL description of the divider.


* Open the file in Vivado, observe the code.

* Draw the RT operations flow diagram (ASMD) and verify that it agrees with the algorithm as described.


______

## Step 2 - Binary/BCD Converter ##

### Step 2.1 ### 

To be able to view a binary number on a display, it is necessary to use a binary to BCD (Binary Coded Decimal) converter, usually implemented as a shift register. The figure below illustrates the process of converting the number 12710 from binary ("1111111") to BCD. The algorithm can be described this way:

1. For each BCD (4-bit) digit in the shift register, check to see if the number is greater than 410. If so, add the number 310.

2. Move the entire record left one position by inserting the MSB of the binary sequence into the LSB of the record.

3. Repeat steps 1 and 2 until all bits of the binary number have been used.


<div>
<img src="lab5/fig2.png" width="500"/>
</div>



* The VHDL description of this circuit is given in **bin2bcd.vhd**. Add it to the Vivado project (*Add or create design sources*). Open the file, look at the code and check that it complies with the algorithm, as described.



* To simulate, synthesize or implement this module, you must first right-click on the module (in the Sources window ), and then choose the **Set as Top** option that appears in the dialog window. This option allows you to define the module as the circuit to be considered in the following actions.

* Create a stimulus file for this module and simulate its operation.


* Add the internal signal **n_reg** to the simulator's waveform window. Set the signal **n_reg** and the outputs **bcd0 , bcd1 , bcd2 and bcd3** to decimal format. To do this, right-click on the signal and choose **{Radix}{Unsigned Decimal}**. Analyze the simulation results.


* Close the Simulation view in the blue strip.


______________

## Step 3 - Digital Frequency Counter for low frequencies  ##

### Step 3.1 ### 

This circuit intends to measure the frequency of a periodic input signal. The simplest way to implement a frequency meter is to use a counter that counts the number of pulses in the input signal during a predefined time (for example, 1 second). Although this solution is sufficient for a high-frequency signal, it does not allow you to accurately measure low-frequency signals. In this case, the best way would be to measure its period and then invert it to obtain the frequency. This is the solution we will implement next.

The circuit will then have to perform the following tasks:

1. Measurement of signal period;

2. Calculation of frequency, performing a division operation (f=1/T);

3. Conversion of the result (frequency in binary representation) to BCD, for display on 7-segment displays. This operation will be performed by the module analyzed in the previous section, defined in the file **bin2bcd.vhd**.



To measure the period of the input signal, we will use a counter that counts the number of clock cycles between two rising edges of this signal. Once the frequency of the clock signal is known (fclk=100MHz), the period of the input signal can be easily obtained. For example, if the circuit counts N cycles, then the period is Tsi=NxTclk and we say that the measurement resolution is Tclk=10ns. If the clock of this counter has a frequency lower than 50MHz, the resolution will necessarily be lower.

The flow diagram (ASMD) is represented in the figure below. When the **start** input is asserted, the circuit waits for the first rising edge of the input signal (**waite state**). This occurrence is signaled by the edge **signal**, generated by an edge detector circuit (discussed in Lab4). As soon as the edge is detected, the circuit resets the **p** and **t** registers and goes to the **count** state, where the temporal measurement is performed. When a second edge is detected, the circuit goes to the **done** state, where the **done_tick** signal is asserted, indicating the end of the measurement. 

Two registers are used. The **t** register counts between 0 and 99999 (100000 clock cycles) with a clock signal at 100MHz while the **p** register is only incremented when **t** reaches the end. This register (p) is therefore incremented with a signal at 100MHz/100000=1kHz, which results in a resolution of 1 millisecond. On the other hand, this is a 10-bit register, so its maximum value is 1023, which corresponds to a maximum period of 1023x1ms=1s (i.e. a minimum frequency of 1Hz). The minimum value is 1, which corresponds to a maximum period of 1ms (i.e. a maximum frequency of 1kHz).

<div>
<img src="lab5/fig3.png" width="400"/>
</div>


* Add **period_counter.vhd** to your project. Analyze the VHDL code. Verify that the value of the **CLK_MS_COUNT** constant has been changed to 10 (instead of 100000) to facilitate simulation verification. 


* Change this constant to a GENERIC, to make it easier to change between simulation and implementation.


* Now add the **slow_freq.vhd** file that describes the circuit shown in the figure below - complete frequency measurement circuit. It integrates the period measurement circuit (**period_counter.vhd**), a divider circuit (**div.vhd**), a binary to BCD converter (**bin2bcd.vhd**), and a state machine that controls the sequence of operations. The ASMD diagram of this controller is shown in the figure.


<div>
<img src="lab5/fig4.png" width="900"/>
</div>


* SSelect the **slow_freq.vhd** file and ensure it is the top file in the *Sources* window. Make the necessary changes to map the **GENERIC CLK_MS_COUNT**.


* Create a stimulus file and check circuit operation. In this file: 1) set the clock period to 100us; 2) define the si signal as a square wave with 2000 clock cycles ON and 2000 cycles OFF. This means that the si signal has a period of Tsi=4000xTclk=400ms. Therefore, its frequency is 2.5Hz. Run the simulation and verify that the circuit works correctly.

* Now increase the frequency of the si signal to 400xTclk => fclk=25Hz (Tsi=40ms) and repeat the simulation. Note that now the output is wrong because the frequency is outside the circuit range (which is 1Hz to 10Hz). Pull the input and output signals from the divider and put them into decimal form. Check that the output is 25641 (25.641Hz), that is, the circuit is measuring the period well, doing the division well, but it does not show the results well as it does not show the most significant digits!

* Now increase the frequency of the si signal to 40xTclk => fclk=250Hz (Tsi=4ms) and repeat the simulation. Note that now the output is wrong but the circuit itself is measuring wrong since the period of the signal to be measured is very close to the resolution of the measurement system (1ms).




_________

## Challenge ##

### Challenge 1 ###
Implement a circuit that allows you to view the decimal value corresponding to an 8-bit binary number from the switch set on the 7-segment displays. You should use the converter circuit tested in step 2 of this work, together with the 7-segment display scanning circuit, analyzed/used in previous works.

### Challenge 2 ###
Create a square wave generator circuit that allows you to test the operation of the frequency meter on the Basys3. The output of the wave generator must be routed to an expansion pin and then enter the board on another expansion pin, corresponding to the frequency meter input. Remember that for implementation, the GENERIC CLK_MS_COUNT must be set to 1000000.

